In [1]:
!CHGCARdataseperate.sh
!ls

The lattice file contain the lattice parameters. The CHGCARclean file only contain the density data very convient for other program to deal with. The NGFS file is NGXF NGYF NGZF
 CHG		     CONTCAR    INCAR	  OSZICAR   POTCAR	  XDATCAR
 CHGCAR		     DOSCAR     KPOINTS   OUTCAR    REPORT
 CHGCARclean	     EIGENVAL   lattice   PCDAT     vasprun.xml
'CHGD(z)get.ipynb'   IBZKPT     NGFS	  POSCAR    WAVECAR


In [2]:
import numpy as np

# Get the lattice parameters

In [3]:
file=open('lattice','r+') #open file lattice---------------------------------------------------------
lattices=file.readlines()
a=np.asarray(lattices[1].strip().split(),dtype='float32')
b=np.asarray(lattices[2].strip().split(),dtype='float32')
c=np.asarray(lattices[3].strip().split(),dtype='float32')
file.close() #close file lattice---------------------------------------------------------------------
al=np.sqrt(np.dot(a,a))
bl=np.sqrt(np.dot(b,b))
cl=np.sqrt(np.dot(c,c))
print(a[0])
Vcell=np.dot(np.cross(a,b),c)
print(Vcell)#print(format(Vcell,'.2f'))

6.899794
1284.0366


# Get the NGXF NGYF NGZF

In [4]:
file=open('NGFS','r+') #open file NGFS---------------------------------------------------------------
xyz=np.asarray(file.readline().strip().split(),dtype='int')
file.close() #close file NGFS---------------------------------------------------------------------

# read the charge density: $\rho(r)\times V_{cell}$

In [5]:
file=open('CHGCARclean','r+') #open file CHGCARclean---------------------------------------------------------
dataoneline=len(file.readline().strip().split())
file.close() #close file CHGCARclean-------------------------------------------------------------------------
file=open('CHGCARclean','r+') #open file CHGCARclean---------------------------------------------------------
CHG=file.readlines()
file.close() #close file CHGCARclean-------------------------------------------------------------------------
print(dataoneline)

5


# calculate the $D(z)$. It will take a lot of time to do this stape

In [6]:
f=open('Dz.dat','w') #open file Dz.dat to write z and D(z)
f.write("z Dz\n")
finishline=10
print("now begin:")
for i in range(0,xyz[2]): #from 0 to xyz[2]-1
    CHGdata=0
    for j in range(0,xyz[1]):
        for k in range(0,xyz[0]):
            n=i*xyz[1]*xyz[0]+j*xyz[0]+k+1
            CHGi=int(np.floor(n/dataoneline))#the line number
            CHGj=np.mod(n,dataoneline)#the column
            if (CHGj==0):
                CHGj=dataoneline
                CHGi=CHGi-1
            CHGj=CHGj-1
            CHGdataline=np.asarray(CHG[CHGi].strip().split(),dtype='float32')
            CHGdata=CHGdataline[CHGj]/xyz[0]/xyz[1]/cl+CHGdata
        #k done
    #j done
    cx=cl*(i+0.5)/xyz[2]
    str1=str(cx)
    str2=str(CHGdata)
    strtot=str1+' '+str2+'\n'
    f.write(strtot)
    if (i>finishline*xyz[2]/100):
        persents=str(finishline)+'%'+'......'
        print(persents)
        finishline=10+finishline
#i done
f.close()
print('job done')

now begin:
10%......
20%......
30%......
40%......
50%......
60%......
70%......
80%......
90%......
job done


# Finally the job done you can use origin to plot the Dz.dat